# Trabajo Práctico N° 2
## Objetivo:

Para cada dispositivo presentado por Jampp, determinar el tiempo que transcurrirá hasta que el mismo aparezca nuevamente en una subasta, y el tiempo hasta que el usuario del mismo decida instalar una nueva aplicación.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import auc, accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
import datetime as dt

## Analizo los datos de la primer ventana

In [2]:
auct = pd.read_csv('data/auctions_ventana1.csv', dtype = {"device_id": 'category', "ref_type_id": np.int8, "source_id": np.int8})

In [3]:
auct.dtypes

date             object
device_id      category
ref_type_id        int8
source_id          int8
dtype: object

In [4]:
auct['date'] = pd.to_datetime(auct['date'])

In [5]:
auct.head()

,date,device_id,ref_type_id,source_id
0,2019-04-20 23:57:27.912838,1109595589636746168,7,0
1,2019-04-20 23:57:28.381114,5896614299191635403,1,0
2,2019-04-20 23:57:28.515423,4172466725848941608,1,0
3,2019-04-20 23:57:28.700884,2616279795187318849,7,0
4,2019-04-20 23:57:28.868312,8034952072073026056,1,0


## Veo cuantas veces aparece cada dispositivo en una subasta

Inicio sencillamente contando la cantidad de subastas en las que participó cada dispositivo, y lo agrego como un nuevo feature

In [6]:
dfApT = auct.groupby('device_id').agg({'date': 'count'}).reset_index()
dfApT.columns = ['ref_hash', 'auctions_count']

In [7]:
dfApT.head()

,ref_hash,auctions_count
0,1000061425870948777,8
1,1000503394293263005,343
2,1001008640113335510,24
3,1001123163431776865,68
4,1001144380199556647,78


## Veo la cantidad de veces que aparece cada dispositivo segun el source_id

Ahora me interesa ver la cantidad de veces que un dispositivo participó en una subasta desde cada tipo de fuente desde donde se produce la subasta. De esta manera los source_id más populares tomaran un valor mayor, luego tomo la desviación estándar de la cantidad para cada device_id

In [8]:
auct['apariciones'] = 1

In [9]:
dfA = auct.groupby(['device_id', 'source_id']).agg({'apariciones':'sum'}).unstack(1).fillna(0).reset_index()
dfA.columns = dfA.columns.droplevel(0)
dfA.columns = ['ref_hash', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
dfA.head()

,ref_hash,0,1,2,3,4,5,6,7,8,9
0,1000061425870948777,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1000503394293263005,225.0,79.0,1.0,18.0,1.0,0.0,0.0,0.0,19.0,0.0
2,1001008640113335510,9.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1001123163431776865,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
4,1001144380199556647,46.0,15.0,1.0,0.0,1.0,0.0,1.0,0.0,14.0,0.0


In [10]:
dfA['auctions_by_srcID'] = dfA.iloc[:,1:].std(axis = 1)
dfA.head()

,ref_hash,0,1,2,3,4,5,6,7,8,9,auctions_by_srcID
0,1000061425870948777,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.201010
1,1000503394293263005,225.0,79.0,1.0,18.0,1.0,0.0,0.0,0.0,19.0,0.0,71.327180
2,1001008640113335510,9.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.253570
3,1001123163431776865,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,19.809089
4,1001144380199556647,46.0,15.0,1.0,0.0,1.0,0.0,1.0,0.0,14.0,0.0,14.649991


## Veo la cantidad de veces que aparece cada dispositivo en una subasta según el ref_type

In [11]:
auct['ref_type_id'].value_counts()

1    13313564
7     2331211
Name: ref_type_id, dtype: int64

In [12]:
dfApRef = auct.groupby(['device_id', 'ref_type_id']).agg({'apariciones':'sum'}).unstack(1).fillna(0).reset_index()
dfApRef.columns = dfApRef.columns.droplevel(0)
dfApRef['median_count_ref_type'] = dfApRef.median(axis = 1)
dfApRef.head()

ref_type_id,,1,7,median_count_ref_type
0,1000061425870948777,8.0,0.0,8.0
1,1000503394293263005,0.0,343.0,343.0
2,1001008640113335510,24.0,0.0,24.0
3,1001123163431776865,68.0,0.0,68.0
4,1001144380199556647,0.0,78.0,78.0


## Cantidad de apariciones de un dispositivo en las subastas por día

Calculo la cantidad de apariciones para cada dispositivo en las subastas por día, luego tomo la mediana para las apariciones.

In [13]:
auct['fecha'] = auct['date'].dt.date

In [14]:
dfApDay = auct.groupby(['device_id', 'fecha']).agg({'apariciones':'sum'}).unstack(1).fillna(0).reset_index()
dfApDay.columns = dfApDay.columns.droplevel(0)
dfApDay.columns = ['ref_hash', '2019-04-18', '2019-04-19', '2019-04-20']
dfApDay['auctions_by_day'] = dfApDay.median(axis = 1)

In [15]:
dfApDay.head()

,ref_hash,2019-04-18,2019-04-19,2019-04-20,auctions_by_day
0,1000061425870948777,2.0,2.0,4.0,3.0
1,1000503394293263005,111.0,99.0,133.0,122.0
2,1001008640113335510,24.0,0.0,0.0,12.0
3,1001123163431776865,16.0,20.0,32.0,26.0
4,1001144380199556647,34.0,27.0,17.0,30.5


## calculo cuanto tiempo tarda un dispositivo en aparecer en una subasta contando desde el inicio de la ventana

La idea es determinar el tiempo que transcurrió entre cada aparición de un dispositivo en una subasta, para luego tomar el tiempo promedio de aparición de un dispositivo en una subasta y de acuerdo a eso predecir.

In [61]:
auct['date_inicial'] = dt.datetime(2019,4, 18)
auct['date_inicial'] = pd.to_datetime(auct['date_inicial'])
auct['timeToAuction'] = (auct['date'] - auct['date_inicial'])/np.timedelta64(1,'s')

In [62]:
auct.head()

,date,device_id,ref_type_id,source_id,apariciones,fecha,date_inicial,timeToAuction
0,2019-04-20 23:57:27.912838,1109595589636746168,7,0,1,2019-04-20,2019-04-18,259047.912838
1,2019-04-20 23:57:28.381114,5896614299191635403,1,0,1,2019-04-20,2019-04-18,259048.381114
2,2019-04-20 23:57:28.515423,4172466725848941608,1,0,1,2019-04-20,2019-04-18,259048.515423
3,2019-04-20 23:57:28.700884,2616279795187318849,7,0,1,2019-04-20,2019-04-18,259048.700884
4,2019-04-20 23:57:28.868312,8034952072073026056,1,0,1,2019-04-20,2019-04-18,259048.868312


Tomo el tiempo mínimo y máximo, en SEGUNDOS, que tardó cada dispositivo en aparecer en una subasta

In [70]:
auction_time = auct.groupby('device_id').agg({'timeToAuction': ['min', 'max']}).reset_index()
auction_time.columns = auction_time.columns.droplevel(1)
auction_time.columns = ['ref_hash', 'minTimeToAuction', 'maxTimeToAuction']

In [71]:
auction_time.head()

,ref_hash,minTimeToAuction,maxTimeToAuction
0,1000061425870948777,13095.182558,240322.786611
1,1000503394293263005,11.398298,254840.374740
2,1001008640113335510,61148.713542,62233.751854
3,1001123163431776865,1442.033706,258920.166523
4,1001144380199556647,1977.283070,258611.272562


## Creo un solo dataframe con los datos estadísticos sobre las subastas

In [55]:
auctions = dfApT.merge(dfA[['ref_hash', 'auctions_by_srcID']], on = 'ref_hash')


In [56]:
auctions = auctions.merge(dfApDay[['ref_hash', 'auctions_by_day']], on = 'ref_hash')
auctions = auctions.merge(auction_time, on = 'ref_hash')
auctions.head()

,ref_hash,auctions_count,auctions_by_srcID,auctions_by_day,timeToAuction
0,1000061425870948777,8,2.201010,3.0,145187.148166
1,1000503394293263005,343,71.327180,122.0,145258.775636
2,1001008640113335510,24,5.253570,12.0,61689.412249
3,1001123163431776865,68,19.809089,26.0,174293.128577
4,1001144380199556647,78,14.649991,30.5,128185.348035


## Clicks

Primero considero el tiempo que tarda cada dispositivo en dar un click para esto utilizo todos los datos sin usar las divisiones en ventanas.

In [72]:
clks = pd.read_csv("data/clks_ventana1.csv", dtype = {'advertiser_id': np.int8, 'action_id': np.float32, 'source_id': np.int8, 'country_codde': 'category',  'carrier_id': np.float16, 'specs_brand': 'category', 'brand': np.float16, 'ref_type': 'category', 'ref_hash': 'category'})

In [73]:
clks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12768 entries, 0 to 12767
Data columns (total 20 columns):
advertiser_id      12768 non-null int8
action_id          1 non-null float32
source_id          12768 non-null int8
created            12768 non-null object
country_code       12768 non-null int64
latitude           12768 non-null float64
longitude          12768 non-null float64
wifi_connection    12768 non-null bool
carrier_id         12548 non-null float16
trans_id           12768 non-null object
os_minor           12761 non-null float64
agent_device       2004 non-null float64
os_major           12761 non-null float64
specs_brand        12768 non-null category
brand              2923 non-null float16
timeToClick        7510 non-null float64
touchX             8571 non-null float64
touchY             8571 non-null float64
ref_type           12768 non-null category
ref_hash           12768 non-null category
dtypes: bool(1), category(3), float16(2), float32(1), float64(8), int6

In [27]:
click_time['possible_click'] = click_time['timeToClick'].transform(lambda x: 1/x if (x > 0) else 0.0).dropna(0.0)

In [28]:
click_time['possible_click'].value_counts()

0.000000    9636
5.000000      11
3.030303      10
5.208333      10
3.424658       9
3.278689       9
4.098361       9
3.436426       9
4.807692       9
2.832861       8
4.504505       8
6.211180       8
3.906250       8
4.651163       8
4.366812       8
2.617801       8
5.714286       8
5.128205       8
3.344482       8
3.610108       7
4.385965       7
4.032258       7
9.090909       7
0.815661       7
3.401361       7
0.843882       7
5.524862       7
3.311258       7
0.816993       7
2.325581       7
            ... 
0.199987       1
0.091777       1
0.239234       1
0.239981       1
0.297841       1
0.000452       1
0.003324       1
0.000806       1
0.000477       1
0.049307       1
0.016814       1
0.001209       1
0.005590       1
0.114541       1
0.483092       1
0.034087       1
0.123747       1
0.024754       1
0.000671       1
0.009631       1
0.046670       1
0.134716       1
0.560538       1
0.014651       1
0.058123       1
0.008749       1
0.080315       1
0.189591      

## Installs

Utilizo el dataframe de instalaciones completo, sin aplicar una ventana, porque por ahora lo que me interesa es calcular el tiempo promedio que tardó cada dispositivo en hacer una instalación, no me interesan las fechas específicas.

In [75]:
inst = pd.read_csv('data/installs_ventana1.csv', dtype = {'application_id': np.int16, 'ref_type': 'category', 'ref_hash': 'category', 'click_hash': 'category', 'device_country_code': 'category', 'device_brand': 'category', 'device_model': 'category', 'kind': 'category', 'device_language': 'category'})

In [76]:
inst['created'] = pd.to_datetime(inst['created'])
inst.head()

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
0,2019-04-20 10:15:36.274,1,1494519392962156891,5230323462636548010,NaN,False,True,6287817205707153877,NaN,8.355495513718673e+18,adjust.com,NaN,dda99e3c-9c4b-487d-891c-79f0a02cb4a8,app_open,NaN,NaN,8291809486355890410,4.060929664968129e+18
1,2019-04-20 21:56:47.151,1,1494519392962156891,5097163995161606833,NaN,False,True,6287817205707153877,NaN,2.3557720913769155e+18,adjust.com,NaN,7010c3ce-0fcf-46c6-9be8-374cc0e20af4,app_open,NaN,NaN,4006811922873399949,3.3013777759776993e+18
2,2019-04-20 22:40:41.239,1,1494519392962156891,6328027616411983332,NaN,False,False,6287817205707153877,NaN,6.156971151807135e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,3386455054590810771,3.3013777759776993e+18
3,2019-04-20 18:19:27.485,1,1494519392962156891,7522785771858684314,NaN,False,True,6287817205707153877,NaN,6.208879341562586e+18,adjust.com,NaN,35e7504f-cd9b-4a78-b89f-7335a8bd735a,app_open,NaN,NaN,7297343258015606683,3.3013777759776993e+18
4,2019-04-20 03:40:21.239,1,1494519392962156891,7882044913917355073,NaN,False,True,6287817205707153877,NaN,4.566898029552894e+18,adjust.com,NaN,f3a8649c-bd47-4874-a20e-8d3d5cedac2e,app_open,NaN,NaN,4764130939738113581,4.060929664968129e+18


In [31]:
inst.dtypes

created               datetime64[ns]
application_id                 int16
ref_type                    category
ref_hash                    category
click_hash                  category
attributed                      bool
implicit                        bool
device_countrycode             int64
device_brand                category
device_model                category
session_user_agent            object
user_agent                    object
event_uuid                    object
kind                        category
wifi                          object
trans_id                      object
ip_address                     int64
device_language             category
dtype: object

## Calculo el tiempo que tarda en convertir cada dispositivo

In [32]:
inst.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481511 entries, 0 to 481510
Data columns (total 18 columns):
created               481511 non-null datetime64[ns]
application_id        481511 non-null int16
ref_type              481511 non-null category
ref_hash              481511 non-null category
click_hash            1142 non-null category
attributed            481511 non-null bool
implicit              481511 non-null bool
device_countrycode    481511 non-null int64
device_brand          276443 non-null category
device_model          454619 non-null category
session_user_agent    466672 non-null object
user_agent            330768 non-null object
event_uuid            103168 non-null object
kind                  103168 non-null category
wifi                  294829 non-null object
trans_id              8933 non-null object
ip_address            481511 non-null int64
device_language       453934 non-null category
dtypes: bool(2), category(7), datetime64[ns](1), int16(1), int64(2),

In [83]:
inst['created_inicial'] = dt.datetime(2019, 4, 18)
inst['created_inicial'] = pd.to_datetime(inst['created_inicial'])
inst['timeToInstall'] = (inst['created'] - inst['created_inicial'])/np.timedelta64(1,'s')
inst.head()

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language,created_inicial,timeToInstall
0,2019-04-20 10:15:36.274,1,1494519392962156891,5230323462636548010,NaN,False,True,6287817205707153877,NaN,8.355495513718673e+18,adjust.com,NaN,dda99e3c-9c4b-487d-891c-79f0a02cb4a8,app_open,NaN,NaN,8291809486355890410,4.060929664968129e+18,2019-04-18,209736.274
1,2019-04-20 21:56:47.151,1,1494519392962156891,5097163995161606833,NaN,False,True,6287817205707153877,NaN,2.3557720913769155e+18,adjust.com,NaN,7010c3ce-0fcf-46c6-9be8-374cc0e20af4,app_open,NaN,NaN,4006811922873399949,3.3013777759776993e+18,2019-04-18,251807.151
2,2019-04-20 22:40:41.239,1,1494519392962156891,6328027616411983332,NaN,False,False,6287817205707153877,NaN,6.156971151807135e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,3386455054590810771,3.3013777759776993e+18,2019-04-18,254441.239
3,2019-04-20 18:19:27.485,1,1494519392962156891,7522785771858684314,NaN,False,True,6287817205707153877,NaN,6.208879341562586e+18,adjust.com,NaN,35e7504f-cd9b-4a78-b89f-7335a8bd735a,app_open,NaN,NaN,7297343258015606683,3.3013777759776993e+18,2019-04-18,238767.485
4,2019-04-20 03:40:21.239,1,1494519392962156891,7882044913917355073,NaN,False,True,6287817205707153877,NaN,4.566898029552894e+18,adjust.com,NaN,f3a8649c-bd47-4874-a20e-8d3d5cedac2e,app_open,NaN,NaN,4764130939738113581,4.060929664968129e+18,2019-04-18,186021.239


Tomo el tiempo máximo y mínimo, en SEGUNDOS, que tardó un dispositivo en realizar una instalación

In [82]:
install_time = inst.groupby('ref_hash').agg({'timeToInstall': ['min', 'max']}).reset_index()
install_time.columns = install_time.columns.droplevel(1)
install_time.columns = ['ref_hash', 'minTimeToInstall', 'maxTimeToInstall']
install_time

,ref_hash,minTimeToInstall,maxTimeToInstall
0,1000289045777700145,181993.347,181993.347
1,100141508580250250,91647.546,91647.546
2,1001467977812205098,219996.582,219996.582
3,1001828092778461413,148749.983,148749.983
4,1001971889699153637,255147.779,255147.779
5,1002823356747674855,188433.643,188433.643
6,1003069378329290746,94604.080,94604.149
7,1003301025712584351,56142.990,56142.990
8,1004091327968593504,255452.078,255452.078
9,1004950562470399111,233921.830,233921.830


## Agrego los dispositivos con installs y el tiempo que tarda cada dispositivo en realizar una instalación

In [38]:
auctions = auctions.merge(install_time[['ref_hash', 'possible_install']], on = 'ref_hash')

In [39]:
auctions = auctions.merge(click_time[['ref_hash', 'possible_click']], on = 'ref_hash')

In [40]:
auctions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11859 entries, 0 to 11858
Data columns (total 7 columns):
ref_hash             11859 non-null object
auctions_count       11859 non-null int64
auctions_by_srcID    11859 non-null float64
auctions_by_day      11859 non-null float64
possible_auction     11859 non-null float64
possible_install     11859 non-null float64
possible_click       11859 non-null float64
dtypes: float64(5), int64(1), object(1)
memory usage: 741.2+ KB


In [41]:
auctions.head()

,ref_hash,auctions_count,auctions_by_srcID,auctions_by_day,possible_auction,possible_install,possible_click
0,1002447145227437936,550,121.294866,193.5,0.000037,0.000000,0.000000
1,1002862509106471090,695,134.424080,302.5,0.000042,0.000000,0.003108
2,1003033929233196737,176,52.879738,72.0,0.000032,0.000000,0.000000
3,1004368020372100067,27,7.165504,13.5,0.000010,0.000000,0.068329
4,1006126025843610775,96,22.741054,40.5,0.000020,1.180638,0.000000


## Pruebo con Gradient Boosting

In [89]:
auctions['label_st'] = auctions['timeToAuction'].astype(int)
auctions.head()

,ref_hash,auctions_count,auctions_by_srcID,auctions_by_day,timeToAuction,label_st
0,1000061425870948777,8,2.201010,3.0,145187.148166,145187
1,1000503394293263005,343,71.327180,122.0,145258.775636,145258
2,1001008640113335510,24,5.253570,12.0,61689.412249,61689
3,1001123163431776865,68,19.809089,26.0,174293.128577,174293
4,1001144380199556647,78,14.649991,30.5,128185.348035,128185


In [ ]:
df_y = auctions['label_st']
df_X = auctions.drop(['ref_hash','timeToAuction', 'label_st'], axis=1)

GBC = GradientBoostingClassifier(random_state=23, n_estimators=50, min_samples_split=50)
scores = cross_val_score(GBC , df_X, df_y, scoring="roc_auc", cv=5)
scores.mean()

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


## Pruebo con XGBoost

In [43]:
''''XGBC = xgb.XGBClassifier(learning_rate =0.075, n_estimators=95, max_depth=4, min_child_weight=6, 
                         gamma=0.3, subsample=0.8, colsample_bytree=0.8, objective= 'linear:logistic',
                         scale_pos_weight=0.8, seed = 15)

scores = cross_val_score(XGBC , df_X, df_y, scoring="roc_auc", cv=5)
scores.mean()'''

'\'XGBC = xgb.XGBClassifier(learning_rate =0.075, n_estimators=95, max_depth=4, min_child_weight=6, \n                         gamma=0.3, subsample=0.8, colsample_bytree=0.8, objective= \'linear:logistic\',\n                         scale_pos_weight=0.8, seed = 15)\n\nscores = cross_val_score(XGBC , df_X, df_y, scoring="roc_auc", cv=5)\nscores.mean()'